In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyarrow.parquet as pq

import my_functions
%load_ext autoreload
%autoreload 2

# Carga de metadata

Se realiza la carga de 'df_id_restaurants' con contiene los identificadores ('gmap_id') de los restaurantes filtrados en el notebook 1.1.

In [2]:
# Cargar el archivo en un DataFrame
df_id_restaurant = pd.read_parquet('Datasets/Google Maps/1_metadata_restaurants/df_id_restaurant.parquet')

In [3]:
my_functions.describir_df(df_id_restaurant)

Cantidad Registros:  210876
Cantidad Campos:  1
Campos:
 Index(['gmap_id'], dtype='object')


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,gmap_id,[<class 'str'>],0,0.0,210876,100.0,210876,100.0


# Carga de datos de Reseñas

La información relativa a las reseñas se encuentra en la carpeta 'reviews-estados' de el repositorio del proyecto. Dentro de la misma, se encuentra una subcarpeta por cada estado de los Estados Unidos y dentro de ellas archivos de tipo JSON que contienen la información fragmentada. 

Comenzaremos por unificar esta la información de cada estado, luego unificar todos los estados y cargarlos en un Dataframe que denominaremos 'df_reviews'.

Ahora bien, solo se tomarán los registros cuyos identificadores de negocios ('gmap_id') se encuentren en 'df_id_restaurant' que contiene exclusivamente establecimientos en cuya categoría se encontró el término 'restaurant' (ver notebook 1.1).

Por otro lado, se extraerán los nombres de los estados en un nuevo campo, 'state_name'.

In [4]:
# Ruta de la carpeta mayor "reviews-estados":
directorio = r"Datasets/Google Maps/reviews-estados"

# Crear DataFrame vacío df_reviews_restaurant
df_reviews = pd.DataFrame()

# Obtenemos la lista de carpetas en la carpeta mayor:
carpeta_mayor = os.listdir(directorio)

for carpeta in carpeta_mayor:

    # Nombramos el nuevo DataFrame con el nombre de la carpeta:
    nombre_carpeta = os.path.basename(carpeta)
    nombre_carpeta = nombre_carpeta.replace("-", "_")  # Reemplazamos guiones por guiones bajos

    # Obtenemos la lista de archivos en la carpeta:
    archivos = os.listdir(f"Datasets/Google Maps/reviews-estados/{carpeta}")

    # Lista para almacenar los DataFrames de los JSON individuales:
    dataframes = []

    # Iteramos sobre los archivos JSON:
    for archivo in archivos:
    
        # Verificamos si el archivo es un archivo JSON:
        if archivo.endswith(".json"):

            # Construimos la ruta completa al archivo:
            ruta_completa = os.path.join(directorio, carpeta, archivo)
            
            # Cargamos el archivo JSON como DataFrame:
            df = pd.read_json(ruta_completa, lines=True)
            
            # Agregamos la nueva columna 'state_name':
            df['state_name'] = nombre_carpeta.replace('review_', '')

            # Agregamos el DataFrame a la lista:
            dataframes.append(df)

    # Concatenamos todos los DataFrames en uno solo
    df_states = pd.concat(dataframes, ignore_index=True)

    # Filtramos registros en df_states que su gmap_id no esté en df_id_restaurant
    df_states_filtered = df_states[df_states['gmap_id'].isin(df_id_restaurant['gmap_id'])]

    # Concatenamos registros filtrados a df_reviews_restaurant
    df_reviews = pd.concat([df_reviews, df_states_filtered], ignore_index=True)

In [5]:
# Se verifica la carga
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11882745 entries, 0 to 11882744
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     float64
 1   name        object 
 2   time        int64  
 3   rating      int64  
 4   text        object 
 5   pics        object 
 6   resp        object 
 7   gmap_id     object 
 8   state_name  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 815.9+ MB


In [6]:
# Se visualiza un registro
df_reviews.head(1)

,user_id,name,time,rating,text,pics,resp,gmap_id,state_name
0,1.179759e+20,Anthony Roberts,1463443013514,4,"On the higher end of price for pizza, but they...",None,None,0x8889221157fb3455:0x5c125c40c3eccc2a,Alabama


Habiendo verificado la carga de datos, se procede a guardar esta información en el archivo 'reviews_restaurants_raw.parquet', en el directorio '2_reviews_restaurants'.

In [7]:
# Nombre de la carpeta a crear
nombre_carpeta = "2_reviews_restaurants"

# Ruta relativa de la carpeta
ruta_carpeta = os.path.join("Datasets", "Google Maps", nombre_carpeta)

# Verificar si la carpeta no existe y crearla si es necesario
if not os.path.exists(ruta_carpeta):
    os.makedirs(ruta_carpeta)
    print(f"Carpeta '{ruta_carpeta}' creada correctamente.")
else:
    print(f"La carpeta '{ruta_carpeta}' ya existe.")


La carpeta 'Datasets\Google Maps\2_reviews_restaurants' ya existe.


In [8]:
# Exportar df_reviews a Parquet
df_reviews.to_parquet('Datasets/Google Maps/2_reviews_restaurants/reviews_restaurants_raw.parquet', index=False)

In [9]:
# Cargar el archivo en un DataFrame
df_reviews = pd.read_parquet('Datasets/Google Maps/2_reviews_restaurants/reviews_restaurants_raw.parquet')

## Descripción general del dataset

## Tipo de datos

Las columnas que presenta nuestro DataFrame son las siguientes:
- user_id: tipo float64, ID del usuario que realiza la reseña.
- name: tipo object, nombre del usuario que realiza la reseña.
- time: tipo int64, tiempo en formato UNIX.
- rating: tipo int64, calificación del negocio del 1(peor) al 5(mejor).
- text: tipo object, texto de la reseña.
- pics: tipo object, imágen de la reseña.
- resp: tipo object, respuesta del negocio a la reseña del usuario, incluyendo tiempo en formato UNIX y texto de la reseña.
- gmap_id: tipo object, ID del negocio.
- state_name: tipo object, nombre del estado.

In [10]:
df_reviews.dtypes

user_id       float64
name           object
time            int64
rating          int64
text           object
pics           object
resp           object
gmap_id        object
state_name     object
dtype: object

Para facilitar el procesamiento se convierten a tipo 'string' los campos de tipo 'object'.

In [11]:
# Modificar columnas a tipo string
df_reviews['name'] = df_reviews['name'].astype('string')
df_reviews['text'] = df_reviews['text'].astype('string')
df_reviews['pics'] = df_reviews['pics'].astype('string')
df_reviews['resp'] = df_reviews['resp'].astype('string')
df_reviews['gmap_id'] = df_reviews['gmap_id'].astype('string')
df_reviews['state_name'] = df_reviews['state_name'].astype('string')

In [12]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11882745 entries, 0 to 11882744
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     float64
 1   name        string 
 2   time        int64  
 3   rating      int64  
 4   text        string 
 5   pics        string 
 6   resp        string 
 7   gmap_id     string 
 8   state_name  string 
dtypes: float64(1), int64(2), string(6)
memory usage: 815.9 MB


Se puede observar que hay más de 11 millones de reseñas que corresponden a los negocios sobre los cuales se aplico el filtro generado a partir de la metadata.

Se ejecuta función para analizar tipo de datos, valores nulos y valores únicos para cada campo.

In [13]:
my_functions.describir_df(df_reviews)

Cantidad Registros:  11882745
Cantidad Campos:  9
Campos:
 Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id',
       'state_name'],
      dtype='object')


,Campo,Tipo de Dato,Valores Nulos,% Nulos,Valores No Nulos,% No Nulos,Valores Únicos,% Únicos
0,user_id,[<class 'float'>],0,0.00,11882745,100.00,5623232,47.32
1,name,[<class 'str'>],0,0.00,11882745,100.00,4137078,34.82
2,time,[<class 'int'>],0,0.00,11882745,100.00,11402264,95.96
3,rating,[<class 'int'>],0,0.00,11882745,100.00,5,0.00
4,text,"[<class 'str'>, <class 'pandas._libs.missing.N...",4809124,40.47,7073621,59.53,5500320,46.29
5,pics,"[<class 'pandas._libs.missing.NAType'>, <class...",11477607,96.59,405138,3.41,376499,3.17
6,resp,"[<class 'pandas._libs.missing.NAType'>, <class...",10596999,89.18,1285746,10.82,1221773,10.28
7,gmap_id,[<class 'str'>],0,0.00,11882745,100.00,127123,1.07
8,state_name,[<class 'str'>],0,0.00,11882745,100.00,51,0.00


## Valores nulos

Se obsevan valores nulos en 3 campos, 'text', 'pics' y 'resp'.

El campo 'text' contiene la reseña escrita por el usuario, información de gran relevancia para nuestro análisis. Casi un 60% de los registros son nulos. A pesar de ello, se decide conservar este campo para su posterior análisis, a la espera de obtener alguna información de valor.

Los campos 'pics' y 'resp' contienen casi el total de datos nulos, por lo cual se decide eliminarlos.

In [14]:
# Eliminar las columnas especificadas
df_reviews = df_reviews.drop(columns = ['pics', 'resp'])

df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11882745 entries, 0 to 11882744
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     float64
 1   name        string 
 2   time        int64  
 3   rating      int64  
 4   text        string 
 5   gmap_id     string 
 6   state_name  string 
dtypes: float64(1), int64(2), string(4)
memory usage: 634.6 MB


A continuación se explora si existen registros duplicados, en cuyo caso deberán ser depurados.

## Registros duplicados

Se ejecuta función para observar cuántas filas están duplicadas y el número total de filas con sus duplicados.

In [15]:
df_muestra_duplicados = my_functions.duplicados(df_reviews)
print("Filas totales con duplicados:", df_muestra_duplicados.duplicated().sum())

Cantidad de registros duplicados:  944331
Filas totales con duplicados: 475668


Hay 946777 registros duplicados. Se eliminarán sus duplicados, conservando uno único de cada uno de ellos.

In [16]:
# Eliminamos los duplicados:
print(f"Cantidad de registros: {len(df_reviews)}")
df_reviews.drop_duplicates(keep='first', inplace=True)
print(f"Cantidad de registros luego del filtro: {len(df_reviews)}")

Cantidad de registros: 11882745


Cantidad de registros luego del filtro: 11407077


Se verifica la eliminación de duplicados.

In [17]:
print("Filas totales con duplicados:", df_reviews.duplicated().sum())

Filas totales con duplicados: 0


## Extracción de año y mes del campo 'time'

Se observa que el campo 'time' contiene datos en formato 'Unix'.

Se extraerá, en primer lugar, la fecha completa en un nuevo campo, 'datetime' y luego se crearán los campos 'year' y 'month' extrayendo información sobre el año y mes de la reseña.

In [18]:
df_reviews['time']

0           1463443013514
1           1447623939865
2           1469293549247
3           1442279219480
4           1382634896130
                ...      
11882740    1512010443190
11882741    1530321971461
11882742    1491401520613
11882743    1556229091462
11882744    1621204235473
Name: time, Length: 11407077, dtype: int64

In [19]:
# Convertir la columna 'datetime' a tipo datetime
df_reviews['datetime'] = pd.to_datetime(df_reviews['time'], unit='ms').dt.date 
df_reviews['datetime'] = pd.to_datetime(df_reviews['datetime'])

# Crear columnas 'year' y 'month'
df_reviews['year'] = df_reviews['datetime'].dt.year
df_reviews['month'] = df_reviews['datetime'].dt.month

# Mostrar el DataFrame con las nuevas columnas
df_reviews[['time', 'datetime', 'year', 'month']].head(3)

,time,datetime,year,month
0,1463443013514,2016-05-16,2016,5
1,1447623939865,2015-11-15,2015,11
2,1469293549247,2016-07-23,2016,7


Se verifica que la conversión se realizó exitosamente y se procede a explorar los valores de año y mes.

In [20]:
# Mostrar valores únicos ordenados en 'year'
valores_unicos_year = sorted(df_reviews['year'].unique())
print("Valores únicos en 'year':", valores_unicos_year)

# Mostrar valores únicos ordenados en 'month'
valores_unicos_month = sorted(df_reviews['month'].unique())
print("Valores únicos en 'month':", valores_unicos_month)

Valores únicos en 'year': [1990, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]
Valores únicos en 'month': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


Las reseñas están distribuídas entre 1999 y 2021, con registros para 1990 que seguramente sean erróneos, considerando la fecha de creación de Google (se explorarán luego para determinar cómo tratarlos).

Tras haber extraído el año y mes de la reseña, se descartan los campos time' y 'datetime' dado que ya no aportan valor.

In [21]:
# Eliminar las columnas especificadas
df_reviews = df_reviews.drop(columns = ['time', 'datetime'])

Se verifican los campos.

In [22]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11407077 entries, 0 to 11882744
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   user_id     float64
 1   name        string 
 2   rating      int64  
 3   text        string 
 4   gmap_id     string 
 5   state_name  string 
 6   year        int32  
 7   month       int32  
dtypes: float64(1), int32(2), int64(1), string(4)
memory usage: 696.2 MB


## Exportar a archivo

Se exporta la información al archivo 'reviews_restaurants_v1.parquet' que se almcenará en la carpeta '2_reviews_restaurants' de el repositorio.

In [23]:
# Guardar el DataFrame concatenado en un nuevo archivo
df_reviews.to_parquet('Datasets/Google Maps/2_reviews_restaurants/reviews_restaurants_v1.parquet')

En el siguiente notebook, "2_EDA_restaurants", se procederá a cruzar la información de la metadata de negocios y de reseñas para poder realizar un análisis exploratorio preliminar.